In [12]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
if __name__ == '__main__':
    scSpark = SparkSession \
        .builder \
        .appName("reading csv") \
        .getOrCreate()

AttributeError: 'Builder' object has no attribute 'encoding'

In [14]:
data_file = '12_19_stt.csv'
sdfData = scSpark.read.csv(data_file, header=True, sep=",", encoding='euc-kr').cache()
print('Total Records = {}'.format(sdfData.count()))
sdfData.show()

Total Records = 9147
+----+---------------+----------+--------+---------------+--------+---------------+------+--------+---------------+
|연도|대상사고 구분명|법정동코드|사고건수|사고건수 구성비|사망자수|사망자수 구성비|치사율|부상자수|부상자수 구성비|
+----+---------------+----------+--------+---------------+--------+---------------+------+--------+---------------+
|2012|     어린이사고|      1101|      37|            0.3|       0|              0|     0|      60|           0.39|
|2012|       노인사고|      1101|     144|           0.51|       4|           0.21|  2.78|     149|            0.5|
|2012|     보행자사고|      1101|     405|           0.79|       9|           0.44|  2.22|     412|            0.8|
|2012|     자전거사고|      1101|      53|           0.41|       0|              0|     0|      55|           0.42|
|2012|       야간사고|      1101|     677|           0.63|       6|           0.21|  0.89|    1015|           0.61|
|2012|     어린이사고|      1102|      28|           0.22|       0|              0|     0|      36|           0.23|
|2012|       

In [15]:
type(sdfData)

pyspark.sql.dataframe.DataFrame

In [16]:
pwd

'/home/pyspark'

In [ ]:
~